In [24]:
from google.cloud import bigquery
import pandas as pd
# Step 2: Set Up BigQuery Client
# Initialize a BigQuery Client
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/moiezismail/Downloads/adidas-API.json"
client = bigquery.Client()
# Initialize a BigQuery Client
client = bigquery.Client()

# U.S. Adidas Sales Analysis

Adidas, a global leader in the sports apparel industry, faces the challenge of understanding its sales dynamics in the competitive U.S. market. This analysis dives deep into Adidas's U.S. sales data, seeking to uncover patterns and insights that can shape future business strategies.

## Exploring Adidas's U.S. Sales

In this project, we will scrutinize Adidas's sales across various U.S. regions, aiming to identify key trends and opportunities for growth. Our analysis will encompass:

- **Exploratory Data Analysis (EDA)**: Examining sales patterns and anomalies.
- **Geo-Spatial Analysis**: Visualizing sales across geographic segments.
- **Dynamic Market Analysis**: Understanding shifts in regional sales performance.

### Key Questions Addressed

1. What are the differences in sales figures and patterns across U.S. regions, states, and cities?
2. How do these trends manifest in a geo-spatial analysis of the data?

### Finance Department: 
- The CFO and regional managers want an initial overview of the company's performance to prepare for the quarterly report.  
- Task: Retrieve total sales and average operating profit by region.

In [25]:
query_1 = """
SELECT region, SUM(Units_Sold) AS total_units_sold, SUM(Sales) AS regional_sales
 FROM adidas123.adidas.adidas
GROUP BY region
ORDER BY regional_sales DESC"""

job_1 = client.query(query_1)
job_1_df = job_1.to_dataframe()
job_1_df

,region,total_units_sold,regional_sales
0,West,224480,10490932
1,Northeast,158199,7162067
2,South,184455,6818931
3,Southeast,114885,5619236
4,Midwest,84692,3438209


#### Observations
- 

### Sales Team
- Adidas wants to reward it's top-performing retailer in 2020.
- Task: Identify the top retailer by total sales, and display how many units it sold

In [49]:
query_2 = """
SELECT Retailer, SUM(units_sold) as untis_sold, SUM(sales) as total_sales
 FROM `adidas123.adidas.adidas`
 WHERE Extract(year from invoice_date) = 2020
 GROUP BY Retailer, units_sold
ORDER BY total_sales DESC
LIMIT 1
"""

job_2 = client.query(query_2)
job_2_df = job_2.to_dataframe()
job_2_df

,Retailer,untis_sold,total_sales
0,West Gear,1470,81060


#### Observation
- 

### Strategy Team
- The team is interested in understanding the growth trajectory for better planning.
- Task: Calculate month-over-month growth in total sales.

In [27]:
query_3 = """
WITH monthly_sales AS (
  SELECT EXTRACT(year from invoice_date) AS year, EXTRACT(month from invoice_date) as month, SUM(sales) as monthly_total_sale
FROM
  `adidas123.adidas.adidas`
GROUP BY year, month
ORDER BY year, month)

SELECT year, month, monthly_total_sale,ROUND(SAFE_DIVIDE(
  (monthly_total_sale - LAG(monthly_total_sale) OVER (ORDER BY year, month)), LAG(monthly_total_sale) OVER (ORDER BY year, month)) * 100,2) AS month_over_month_growth 
FROM monthly_sales
ORDER BY year, month"""

job_3 = client.query(query_3)
job_3_df = job_3.to_dataframe()
job_3_df

,year,month,monthly_total_sale,month_over_month_growth
0,2020,1,763746,NaN
1,2020,2,712238,-6.74
2,2020,3,786827,10.47
3,2020,4,813756,3.42
4,2020,5,525514,-35.42
5,2020,6,223569,-57.46
6,2020,7,519763,132.48
7,2020,8,726480,39.77
8,2020,9,596936,-17.83
9,2020,10,383269,-35.79


#### Observations
- 

### Retail Operations
- The operations team is reviewing staffing needs and wants to know if weekends require more staff based on sales.
- Task: Compare average sales on weekdays versus weekends.

### Business Task: Dynamic Regional Sales Performance Analysis
- Objective: The strategy team aims to evaluate and track the sales performance across different regions over time. They are particularly interested in identifying the regions with the most significant growth in sales each month.

- Task: Analyze monthly sales data to calculate the month-over-month percentage change in sales for each region. Identify the top-performing region each month based on the highest sales growth percentage.

In [28]:
query_5 = """
WITH ct1 AS (SELECT region, extract(year from invoice_date) as year, extract(month from invoice_date) as month, SUM(sales) AS monthly_sales, 
  FROM `adidas123.adidas.adidas`
  GROUP BY region, year, month
  ORDER BY year, month, monthly_sales DESC), 
  ct2 AS (
    SELECT year, month, region, monthly_sales, SAFE_DIVIDE(
      monthly_sales - LAG(monthly_sales) OVER (PARTITION BY region ORDER BY year, month),
      LAG(monthly_sales) OVER (PARTITION BY region ORDER BY year, month)
    ) * 100 AS change_percentage
    FROM ct1),
    ct3 AS (SELECT year, month, region, change_percentage, RANK() OVER (PARTITION BY year, month ORDER BY change_percentage DESC) AS growth_rank
FROM ct2
ORDER BY year, month, growth_rank)

SELECT year, month, region, ROUND(change_percentage) AS change_percentage
FROM ct3
WHERE growth_rank = 1 AND change_percentage IS NOT NULL
ORDER BY year, month, growth_rank"""

job_5 = client.query(query_5)
job_5_df = job_5.to_dataframe()
job_5_df

,year,month,region,change_percentage
0,2020,2,West,77.0
1,2020,3,Northeast,105.0
2,2020,4,Northeast,-15.0
3,2020,5,West,-10.0
4,2020,6,West,-1.0
5,2020,7,West,46.0
6,2020,8,South,371.0
7,2020,9,Southeast,138.0
8,2020,10,South,-17.0
9,2020,11,Midwest,240.0


### Marketing Team
- The team is planning a new ad campaign and wants to align it with customer preferences.
- Task: Find the most popular sales method (Online, In-store, Outlet) by region.

In [29]:
query_6 = """
with cte1 AS (SELECT region, method, COUNT(Method) AS freq_of_method
FROM `adidas123.adidas.adidas`
GROUP BY region, Method),
cte2 AS (SELECT region, method,rank() OVER(PARTITION BY region ORDER BY freq_of_method DESC) AS most_pop_method
FROM cte1)

SELECT region, method
FROM cte2
WHERE most_pop_method = 1"""

job_6 = client.query(query_6)
job_6_df = job_6.to_dataframe()
job_6_df

,region,method
0,South,Online
1,South,Outlet
2,Midwest,Online
3,Northeast,Online
4,West,Online
5,Southeast,Online


### Finance & Product Teams
- Both departments are interested in the profitability of individual products for future planning.
- Task: For each product, calculate the profit margin and rank them.

In [30]:
query_7 = """
WITH cte1 AS (
  SELECT prod_cat, ROUND((Oper_prof / sales ) * 100) AS profitability
  FROM adidas123.adidas.adidas),
  cte2 AS (
    SELECT prod_cat, profitability, RANK() OVER(PARTITION BY prod_cat ORDER BY profitability DESC) AS rank_prof
    FROM cte1
  )

  SELECT DISTINCT(prod_cat) as product_category, profitability
  FROM cte2
  WHERE rank_prof = 1
  ORDER BY profitability DESC"""

job_7 = client.query(query_7)
job_7_df = job_7.to_dataframe()
job_7_df

,product_category,profitability
0,Women's Apparel,80.0
1,Men's Apparel,77.0
2,Women's Athletic Footwear,75.0
3,Men's Street Footwear,70.0
4,Men's Athletic Footwear,65.0
5,Women's Street Footwear,64.0
